In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004034280776977539
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 59.730148949328736
200000 116.8993869218395
300000 113.54689988035943
400000 118.73359052444943
500000 120.160675326501
600000 124.36355407324484
700000 113.60829823297749
800000 116.02095632650351
900000 112.49446368017072
1000000 109.35177329961166
1100000 101.93095346316814
1200000 26.378918087994247
1300000 96.4067158520004
1400000 99.80871780842651
1500000 93.18571586835988
1600000 100.68116855005594
1700000 94.49465154974688
1800000 106.3071282327669
1900000 94.75353407819236
2000000 102.72382167902215
2100000 95.3471319858031
2200000 103.01924016407378
2300000 74.93147311688504
2400000 46.112196831088
2500000 100.6005541537522
2600000 92.65097272019153
2700000 99.76306506338724
2800000 93.19123069482517
2900000 101.18003623636093
3000000 95.93557

29500000 101.68939272371126
29600000 104.73286062028315
29700000 68.96072590619794
29800000 80.11633491385588
29900000 106.08974561896082
30000000 106.42239261256397
30100000 102.48866665848772
30200000 42.66776238049344
30300000 109.51165878747115
30400000 104.07777082673334
30500000 109.84260536130733
30600000 98.64802601876058
30700000 107.93316752188045
30800000 98.28706583229207
30900000 104.36699501208483
31000000 93.40322616094265
31100000 27.12963010817099
31200000 105.70668239825902
31300000 95.44942649569417
31400000 104.21739043903223
31500000 100.32406940076476
31600000 106.16208824384019
31700000 100.20388679559096
31800000 108.26823623485798
31900000 99.80737615060544
32000000 63.40959930385071
32100000 88.57011254540673
32200000 112.55658860796055
32300000 98.41870735987519
32400000 108.25929211207179
32500000 33.898726619996935
32600000 102.62632952115565
32700000 108.87336891072728
32800000 98.69169624721296
32900000 108.27744322955704
33000000 101.90102494655555
33100

59600000 102.49203946804361
59700000 101.01236433766177
59800000 104.14554615432935
59900000 96.91709901235181
60000000 97.63674574757546
60100000 102.43321155168
60200000 99.48852843109847
60300000 39.42065712015071
60400000 95.73270386389235
60500000 92.6182748991143
60600000 90.87440677243616
60700000 88.85119507095061
60800000 87.37455130608647
60900000 87.31891688049411
61000000 24.040982115609058
61100000 94.72697049629406
61200000 86.34143269775551
61300000 88.15453220038295
61400000 86.13518879773754
61500000 92.69789334497423
61600000 93.17990672465803
61700000 37.495673562693526
61800000 103.34178993924225
61900000 100.64952445924375
62000000 96.82080143559098
62100000 102.06084670209717
62200000 100.3411218310032
62300000 99.56708229003874
62400000 102.35098186514914
62500000 100.21614181713346
62600000 51.32124704816204
62700000 84.40416847098886
62800000 95.85877208135473
62900000 93.17409679040004
63000000 85.58110197052173
63100000 90.81698456471132
63200000 91.858214968

90000000 90.2164788271221
90100000 81.76021297000848
90200000 86.41761777296196
90300000 81.77183997620108
90400000 49.330300100505625
90500000 64.52196567085213
90600000 75.42059562315602
90700000 79.5476921700785
90800000 82.40825848024996
90900000 20.458365012953667
91000000 79.63452943429479
91100000 71.2212852360386
91200000 71.94712572826722
91300000 68.07052941738166
91400000 40.32964783081701
91500000 68.79352603133914
91600000 77.9009086709428
91700000 86.06648343613188
91800000 81.39525823017391
91900000 81.53353029069947
92000000 76.1768851797182
92100000 77.19681980130262
92200000 71.876630329948
92300000 73.8518788044556
92400000 85.80847136852631
92500000 81.82528420369839
92600000 74.40316629509587
92700000 55.50889549753776
92800000 47.204288476202784
92900000 68.08638024971582
93000000 79.70437481885283
93100000 69.91281142324735
93200000 22.11813896072003
93300000 74.91175874858656
93400000 74.18258356058602
93500000 78.46653507418031
93600000 45.527481704766984
93700

119600000 73.33778037027376
119700000 75.94920513700619
119800000 65.39372624363736
119900000 42.7263313670827
120000000 41.72191120497918
120100000 40.606834459690305
120200000 60.0704737559987
120300000 43.645565369716344
120400000 41.3482522972619
120500000 31.002564982964465
120600000 23.8602653848537
120700000 41.26416138175687
120800000 10.653323330007638
120900000 62.52691491482153
121000000 39.44999307782684
121100000 29.934985785052096
121200000 42.021023395905765
121300000 38.896306665129856
121400000 41.27645696921864
121500000 39.84765164916019
121600000 61.45496905695495
121700000 45.87999734122797
121800000 42.32126705257535
121900000 48.02942416204549
122000000 59.13025138174016
122100000 45.27759276972271
122200000 42.57801728918688
122300000 40.195149284666904
122400000 45.60217811195913
122500000 61.630455437089886
122600000 40.561026010442006
122700000 62.65194623014912
122800000 54.79503583958529
122900000 18.40331413458531
123000000 68.13942956855348
123100000 12.5

...

1190800000 2.868177468885819
1190900000 2.6584007667139207
1191000000 3.1002580970775786
1191100000 3.443826982779248
1191200000 2.9495917428719935
1191300000 3.3177326121591326
1191400000 3.354048030624963
1191500000 3.0836481691242588
1191600000 3.890116799114287
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.438630575409366
1191800000 1.5670778349858987
1191900000 1.1350819446358382
1192000000 1.6387823018747338
1192100000 1.9104193776820468
1192200000 1.2464900507651369
1192300000 1.4555791931038133
1192400000 2.0322176462570694
1192500000 1.5558657512231668
1192600000 1.3541258045759101
1192700000 2.029906760406683
1192800000 1.5518060709292887
1192900000 1.1141713394035497
1193000000 1.5534674056436548
1193100000 1.478802624418199
1193200000 1.2989124115305213
best so far: 0
type: [1, 4, 1, 11, 3] 132
cases of this type: 431244
1193300000 1.8072665958074232
1193400000 1.9596759165072741
1193500000 1.8057929682314826
1193600000 1.754078145084623

1204500000 3.2181019373393354
1204600000 3.08149061195299
1204700000 2.437946930196709
1204800000 3.2363198762882157
1204900000 2.3813978718002837
1205000000 2.5368786267721357
1205100000 3.1722913228789373
1205200000 2.188131723010103
1205300000 2.7804346545287206
1205400000 3.4731242773657147
1205500000 2.202918148737318
1205600000 2.7507206433217593
1205700000 3.2196473927057085
1205800000 1.9806804431883591
1205900000 3.1814376593403417
1206000000 3.0435881070591013
1206100000 1.9201610093832846
1206200000 3.213355990601798
1206300000 3.0633757786339224
1206400000 1.6467341258413197
1206500000 3.1614594786565005
1206600000 2.879753701154729
1206700000 2.2397605971115
1206800000 3.129072250026348
1206900000 3.1141568313294354
1207000000 2.565989890364044
1207100000 2.016255312800573
1207200000 0.5122563212994934
1207300000 0.5110710366850065
1207400000 0.5349848592498833
1207500000 0.5255832213183642
1207600000 0.5332225194140805
1207700000 0.5203113740167784
1207800000 0.4682925774

1231500000 0.7310850152260662
1231600000 0.6190072659772138
1231700000 0.5193606953578028
1231800000 0.6983027501659591
1231900000 0.7495404975541399
1232000000 0.48547172820455164
1232100000 0.477426212583989
1232200000 0.37878864179116817
1232300000 0.32759133596983553
1232400000 0.5811238694911599
1232500000 0.6347251135323808
1232600000 0.6000457603695061
1232700000 0.6806813083065351
1232800000 0.565214968488627
1232900000 0.7032247784261306
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 0.7846193706038893
1233100000 0.4456589520750774
1233200000 0.6386642761648893
1233300000 0.7315347512880565
1233400000 0.24745773426195647
1233500000 0.6989699564822647
1233600000 0.615355309360534
1233700000 0.2822180330871145
1233800000 0.6111658666321701
1233900000 0.6550078171802759
1234000000 0.4290239748862088
1234100000 0.7209756434812779
1234200000 0.4001158583495915
1234300000 0.4801828545482026
1234400000 0.6339770030737387
1234500000 0.540504350552708


1252900000 0.2729533068814344
1253000000 0.37682480975155697
1253100000 0.1838283930834532
1253200000 0.31168314686255655
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.2107847810336418
1253400000 0.12061323862596353
1253500000 0.10097674756913715
1253600000 0.11999834808681077
1253700000 0.10796708861509959
1253800000 0.12814077660906315
1253900000 0.11290267491897941
1254000000 0.09348519481578468
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.1232259135802918
1254200000 0.14181795949800807
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.10547151399408779
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.11108140861325794
1254500000 0.05908612869120969
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of this type: 23958
best so far: 0
type: [2, 2, 11, 1, 3] 132
cases of this type: 19602
1254600000 0.10920736928725243
best so far: 0
type: [2, 2, 11, 3, 

best so far: 0
type: [3, 4, 1, 1, 11] 132
cases of this type: 58564
best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best so far: 0
type: [3, 44, 1, 1, 1] 132
cases of this type: 44
best so far: 0
type: [4, 1, 1, 1, 33] 132
cases of this type: 1185921
1266100000 0.021654396134624877
1266200000 0.03918799169196354
126630